In [1]:
import pandas as pd
import matplotlib.pyplot as plt

This algorithm was created to check whether a product intended for human consumption has been cooked at the correct temperature and for the correct time to prevent food poisoning.

This is a Critical Control Point (CCP) - compliance with established regulations is a legal requirement and a commitment to consumers’ well-being, making CCPs a cornerstone in the global food industry.

For more details, visit: https://www.food.gov.uk/safety-hygiene/cooking-your-food

AS IS PROCESS Cooking data is downloaded manually from data loggers as an excel file and saved in a folder. It is important to use the correct naming convention as this will ensure clarity in the traceability information. Then the algorithm loads the cooking data.

Naming Convention - CookingNumber_ProductID_ChamberNumber_DataLoggerPosition(Top,Middle,Bottom)

Cooking/ Cooling temperatures and times requirements

Equivalent process is an important term to understand when determining what temperature (and time at that temperature) is required when re-heating foods. The usual thermal process required when re-heating foods is to attain and hold a temperature of 70°C for two minutes. However, an equivalent process to this can be achieved by higher temperature and shorter times or by lower temperatures and longer times. There is a method to calculate this and the table below outlines the different time and temperature combinations calculated to achieve the same (equivalent) thermal process to two minutes at a temperature of 70°C.

! For quality reasons, temperature higher than 70°C is not permitted.

In [2]:
# Table of cooking temperature and time requirements
cooking_combinations = pd.DataFrame({
    'Temp [°C]': [70, 65, 60], 
    'Time [min]': [2, 10, 45], 
    })

# Table of cooling temperature and time requirements
cooling_combinations = pd.DataFrame({
    'Temp [°C]': [4, 5],
    'Time [min]': [240, 300],
    })

In [3]:
# File name 
file_path = "20K30_12348_CH2_TOP"+".xlsx" 

# Create a DataFrame from the provided file
cooking_data = pd.read_excel(file_path)

# Create a list with names of data frame columns
columns_names = list(cooking_data.columns)

In [4]:
# Data is downloaded from oldest to newest. Data reversing required
cooking_data = cooking_data.loc[reversed(cooking_data.index)].reset_index(drop=True)

# This variable verifies whether the data verification process is approved
quality_check = "Pass" 

# Check if there is any missing date or temperature data.
# If so, stop the process and inform the Quality Manager immediately.

time_column_cleaning = cooking_data["Time"].isnull().sum()
temp_column_cleaning = cooking_data["Temp"].isnull().sum()

if time_column_cleaning > 0:
    quality_check = "Fail" # Set Quality Check to FAIL
    print("Missing Time data")
    print(cooking_data.to_string()) # Print row data so that the auditor can visually verify cooking
    
if temp_column_cleaning > 0:
    quality_check = "Fail"
    print("Missing Temp data")
    print(cooking_data.to_string()) # Print row data so that the auditor can visually verify cooking

In [5]:
if quality_check == "Pass": # Start only if quality check passed
    
    core_temp = ""
    cooking_validation = ""
    
    for i in cooking_combinations.index: # Loop 1
        
        # This Loop allows to iterate through the cooking_combinations table and
        # runs until end of the index or validation is passed. 
        
        x = len(cooking_data)-1 # Reset "x" to start looping from the beginning
       
        while True: # Loop 2; repeat forever unless it reaches "break" 
                    
            # It is possible that there will be several points that reached
            # the core temp but not for the required amount of time. So, if 
            # the first period of cooking time found is failed, the algoritms 
            # check the next one at so on until  x = 0 or validation is passed. 
           
            while x >= 0: # Loop 3; Find tima of end cooking
                if cooking_data.loc[x, columns_names[1]] >= cooking_combinations.loc[i, 'Temp [°C]']:
                    cooking_time_end = cooking_data.loc[x, columns_names[0]]
                    core_temp = cooking_combinations.loc[i, 'Temp [°C]']
                    break # finished; exit the loop 3
                x -= 1
            
            if x + 1 == 0:
                break # finished; exit the loop 2
            
            while x >= 0:  # Loop 4; Find tima of start cooking
                if cooking_data.loc[x, columns_names[1]] < cooking_combinations.loc[i, 'Temp [°C]']: 
                    cooking_time_start = cooking_data.loc[x+1, columns_names[0]]
                    break # finished; exit the loop 4
                x -= 1
            
            if x + 1 == 0:
                break # finished; exit the loop 2
            
            # Calculates cooking time
            total_cooking_time_in_min = round((cooking_time_end - cooking_time_start).total_seconds() / 60)
            
            # Checks if the cooking time is long enough
            if total_cooking_time_in_min >= cooking_combinations.loc[i, 'Time [min]']:
                control_cooking_temp = cooking_combinations.loc[i, 'Time [min]']
                cooking_validation = "Pass"
                break # finished; exit the loop 2

        if cooking_validation == "Pass":
            break  # finished; exit Loop 1

In [6]:
# Print result of cooking validation
if cooking_validation == "Pass":
    print("Pass Cooking Validation")
elif core_temp == "":
    print("Cooking data invalid. Core temp has not been found")
elif cooking_validation == "" and core_temp != "":
    print("Cooking time failed - too short")  

Pass Cooking Validation


In [7]:
if cooking_validation == "Pass": # Start only if cooking validation passed
    
    cooling_time_start = ""
    cooling_time_end = ""
    result = "Fail"
    
    search_cooling_time_until = x # This ensures that only post-cooking time interval will be analysed.

    for j in cooling_combinations.index: # Loop 1
        
        # This Loop allows to iterate through the cooling_combinations table and
        # runs until end of the index or cooling validation is passed 

        while x <= len(cooking_data)-1: # Loop 2; Find tima of start cooling. 
            if cooking_data.loc[x, columns_names[1]] <= 63: 
                cooling_time_start = cooking_data.loc[x, columns_names[0]]
                break # finished; exit Loop 2
            x += 1

        x = len(cooking_data)-1 # Reset "x" to start looping from the beginning

        while x >= search_cooling_time_until: # Loop 3; Find tima of end cooling. 
            if cooking_data.loc[x, columns_names[1]] <= cooling_combinations.loc[j, 'Temp [°C]']: 
                cooling_time_end = cooking_data.loc[x, columns_names[0]]
                break # finished; exit Loop 3
            x -= 1

        if cooling_time_end == "": 
            continue # Quick check to find if fisrt core temp has been found, if no then go to next i
        
        # Calculates cooling time
        total_cooling_time_in_min = round((cooling_time_end - cooking_time_start).total_seconds() / 60,2)  

        # Checks if the cooling time is long enough
        if total_cooling_time_in_min <= cooling_combinations.loc[j, 'Time [min]']:
            control_cooling_time = cooling_combinations.loc[j, 'Time [min]']
            print("Pass Cooling Validation at", cooling_combinations.loc[j, 'Temp [°C]'], "°C")
            result = "Pass" # The final result for whole validation
            break # finished; exit Loop 1

Pass Cooling Validation at 5 °C


In [8]:
# Find postions of all "_" in the file name. This allows you to extract the following data
positions = [i for i, char in enumerate(file_path) if char == "_"]

# Set traceability information. To be extracted from the file name
cooking_number = file_path[:positions[0]] 
product_code = file_path[positions[0]+1:positions[1]]
chamber_no = file_path[positions[1]+1:positions[2]]
logger_position = file_path[positions[2]+1:file_path.find(".")]

cooking_date = str(cooking_time_start)[:10]

import os 
auditor = os.getlogin() # Set auditor details

# Set date and time of cooking validation
import datetime 
auditing_date = str(datetime.datetime.now())[:10]
auditing_time = str(datetime.datetime.now())[11:16]

# Create Data Frame from cooking validation results
new_data = pd.DataFrame({
    "auditing_date":[auditing_date],
    "auditing_time":[auditing_time],
    "cooking_number":[cooking_number],
    "cooking_date": [cooking_date], 
    "product_code": [product_code],
    "chamber_no":[chamber_no],
    "logger_position": [logger_position], 
    "core_temp": [core_temp],
    "control_cooking_temp":[control_cooking_temp],
    "total_cooking_time_in_min":[total_cooking_time_in_min],
    "control_cooling_time": [control_cooling_time], 
    "total_cooling_time_in_min": [total_cooling_time_in_min],
    "result": [result]
    })

In [9]:
file_path1 = "BazaDanych.xlsx"

# Read the existing Excel file
data_base = pd.read_excel(file_path1,sheet_name="Cooking Results")

# Append the new data (cooking validation results) to the existing DataFrame
df_combined = pd.concat([data_base, new_data]).reset_index(drop=True)

# Save the updated DataFrame back to the Excel file
# use `with` to avoid other exceptions
with pd.ExcelWriter(file_path1, mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:
    df_combined.to_excel(writer, sheet_name="Cooking Results",index=False)
    worksheet = writer.sheets["Cooking Results"]
    writer.close

    print("Data added successfully!")

Data added successfully!
